# Commonly Used Loss Functions

The model paramters of Neural Networks are trained using a process known as the back-propagation of errors from a loss function. So model training requires that you choose a loss function when designing your model. 

In this chapter, we'll walk through a few commonly used loss function available in PyTorch and what's involved in the computation. 

At a basic level, Loss Functions can be categorized into two categories: Regression Loss and Classification Loss. 

Below we show examples of both.


In [3]:
import torch
import torch.nn as nn

print(f'Torch version: {torch.__version__}')

Torch version: 1.7.1


## Regression Analysis with a Regression Loss Function
Regression analysis predicts a continuous output variable based on the values of one or more input variables. Briefly, the goal of a regression model is to build a mathematical approximation that defines the output variable as a function of the input variables. An example of a continuous variable is the predicted price of a home based on inputs such as living space, number of rooms, etc. Note, the output is a continuous value, not a discrete value or a categorical value. The input variable can be continuous or discrete.

The loss fucntion associated with regression analysis is known as the regression loss. Mean Square Error (MSE) is the most commonly used regression loss function (and we'll illustrate that here with an example.

MSE is calculated as the mean of squared difference between predictions and actual observations (also know as the target). Expressed as a computation, it is:

`sum( square( predicted_vals - targets ) ) / (number of training samples)` and you'll see it expressed as PyTorch code below.

Intuitively, the squaring of the error term means that larger deviations from the target values amplify the error way more than than smaller deviations, meaning that the model is penalized in a square relationship for deviating from the target values.

A disadvantage of MSE is that a few (or even one) outlier data points can make the loss unnecessarily large and that maynot be desirable since the squared outlier data point is penalizing the entire mean calculation. 


In [11]:
predicted_vals= torch.randn(3, 5, requires_grad=True)
targets = torch.randn(3, 5)
regression_mse_loss = nn.MSELoss()(predicted_vals, targets)  # calculates mean
print('predicted_vals:\n\t', predicted_vals)
print('targets:\n\t', targets)
print('regression loss:\n\t', regression_mse_loss)
hand_calc_regress_mse_loss = torch.sum(torch.square(predicted_vals - targets)) / (targets.shape[0] * targets.shape[1])
print('hand calculated regression loss:\n\t:', hand_calc_regress_mse_loss)
assert torch.equal(regression_mse_loss, hand_calc_regress_mse_loss)

predicted_vals:
	 tensor([[-1.4814,  0.4830,  0.7498, -0.1266, -0.2733],
        [-0.2085,  3.2570, -1.7418, -1.7435, -0.0311],
        [ 0.8718,  0.6493, -0.7154,  1.9743, -0.9428]], requires_grad=True)
targets:
	 tensor([[ 1.3720,  0.4140,  0.8847, -0.7241,  0.1338],
        [ 0.1846, -1.5605,  0.6315,  0.0645, -1.3102],
        [ 0.7751,  0.2462,  0.1291, -0.1041,  0.4175]])
regression loss:
	 tensor(3.3095, grad_fn=<MseLossBackward>)
hand calculated regression loss:
	: tensor(3.3095, grad_fn=<DivBackward0>)


## Binary Classification with a Cross Entropy Loss Function

Measuring the Binary Cross Entropy Loss between the target and the predicted output is done with: 

`loss = -1 * (target * log(predicted)  +  (1 - target) * log(1 - predicted))` where log means log-base-e

We show below with a simple example that the Cross Entropy Loss defined above penalizes misclassifications more, a property that can help in better learning.

Assume that the target value is 1. We compute the MSE Loss and the Cross Entropy Loss for a correctly predicted value (0.95) and a wrongly predicted value (0.1), i.e., a misclassification.

First, we calculate the MSE loss.

| Target &nbsp &nbsp     | Predicted &nbsp &nbsp| MSE Loss | &nbsp &nbsp Comment | 
| :---:        |    :----:   |          :---: | :--- |
| 1      | 0.95       | (1.0 - 0.95)**2 = 0.0025   | &nbsp &nbsp For a correct prediction, loss is small |
| 1   | 0.1        | (1.0 - 0.1)**2 = 0.81      | &nbsp &nbsp For a wrong prediction, loss is big

For the Cross Entropy Loss, because the target value is 1, the computation reduces to `-1 * target * log(predicted)` or simply `-1 * log(predicted)`.

| Target &nbsp &nbsp | Predicted &nbsp &nbsp| Cross Entropy Loss| &nbsp &nbsp Comment | 
| :---:        |    :----:   |          :---: | :--- |
| 1      | 0.95       | -1 * log(0.95) =  0.051  | &nbsp &nbsp Prediction is correct; loss is small |
| 1   | 0.1        | -1 * log(0.1) = 2.302      | &nbsp &nbsp Prediction is wrong; loss is LARGE when compared to the MSE loss

If the predicted output is close to the desired output, then the loss is small (for both loss functions). The difference is noticeable however when the output is misclassified (0.81 for MSE Loss and 2.302 for Cross Entropy Loss). 

The Cross Entropy Loss function also has the benefit of learning at a faster pace. To learn (the process of continously updating model parameters), we back-propagate the loss, and that is done by taking the partial derivative of the loss with respect to the weights. In doing so, we can show that the rate at which the model parameters (weights and biases) learn is proportional to `output - target`, i.e., proportional to the error in the output. The larger the error, the faster the model will learn which is a very nice property.

In the example below, we create two clusters of normalized data points, 1.5 units apart. Each cluster belongs a class; so we have two classes, class 0 and class 1. The loss computation uses `nn.BCELoss`, the PyTorch Binary Cross Entropy Loss function.

It is typical to use the `nn.Sigmoid` activation as the final output of your neural net followed by the nn.BCELoss to calculate the Binary Cross Entropy loss.

We also calculate is by hand and ensure we get the same results.



In [12]:
input = torch.cat([torch.randn(3), torch.randn(3) + 1.5])  # two clusters 1.5 units apart
target = torch.cat([torch.zeros(3), torch.ones(3)])  # ... from two classes; class 0, class 1
predicted = torch.sigmoid(input)

unreduced_loss = nn.BCELoss(reduction='none')(predicted, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('input:\n\t', input)
print('target:\n\t', target)
print('predicted:\n\t', predicted)
print('unreduced loss:\n\t', unreduced_loss)
print('mean loss:\n\t', mean_loss)

hand_calc_loss = -1 * (target * torch.log(predicted) + (1 - target) * torch.log(1 - predicted))
print('hand calculated loss\n\t', hand_calc_loss)
assert torch.equal(hand_calc_loss, unreduced_loss)  # check they are the same

input:
	 tensor([-1.0610,  0.0233,  1.1670,  3.3213,  0.8761,  2.4037])
target:
	 tensor([0., 0., 0., 1., 1., 1.])
predicted:
	 tensor([0.2571, 0.5058, 0.7626, 0.9652, 0.7060, 0.9171])
unreduced loss:
	 tensor([0.2972, 0.7048, 1.4380, 0.0355, 0.3481, 0.0865])
mean loss:
	 tensor(0.4850)
hand calculated loss
	 tensor([0.2972, 0.7048, 1.4380, 0.0355, 0.3481, 0.0865])


In the exampe above, the Cross Entropy Loss calculation was done in two steps.

1. A Sigmoid lon-linearity on the output

2. Application of the Binary Cross WEntropy Loss

For numerical stability reasons, the two steps above can be collapsed into one step with the `nn.BCEWithLogitsLoss` loss function.

`nn.BCEWithLogitsLoss` loss combines a Sigmoid layer and the BCELoss in one single class

In the example below, we reuse the input tensor from the code above and show the equivalence.


In [6]:
# inputs and target same as above
loss_unreduced = nn.BCEWithLogitsLoss(reduction='none')(input, target)
loss_mean = nn.BCEWithLogitsLoss(reduction='mean')(input, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)  # will be the same as the two step calculation

unreduced loss:
 tensor([0.8812, 0.3035, 1.2865, 0.0443, 0.4270, 0.4390])
mean loss:
 tensor(0.5636)


## Multi-Class Classification with a Cross Entropy Loss Function

We use this loss function when we train a model to output a probability over multiple classes(multi-class classification). 

To be concrete, let's ay we're classifying images of digits. It is used for .

It is useful when training a classification problem with C classes.
The `target` that this loss expects should be a class index in the range :math:`[0, C-1]` where `C = number of classes`; 

References
https://forums.fast.ai/t/nllloss-implementation/20028/

https://gombru.github.io/2018/05/23/cross_entropy_loss/

https://www.youtube.com/watch?v=7q7E91pHoW4&ab_channel=PythonEngineer




In [7]:

input = torch.randn(7, 5)
target = torch.randint(5, (7,))
# predicted = torch.softmax(input, dim=-1)
predicted = nn.LogSoftmax()(input)
unreduced_loss = nn.NLLLoss(reduction='none')(predicted, target)
mean_loss = nn.NLLLoss(reduction='mean')(predicted, target)
print('input:\n\t', input)
print('target:\n\t', target)
print('predicted:\n\t', predicted)
print('unreduced loss:\n\t', unreduced_loss)
print('mean loss:\n\t', mean_loss)

def coded_nll_loss(logs, targets):
    out = torch.zeros_like(targets, dtype=torch.float)
    for i in range(len(targets)):
        out[i] = logs[i][targets[i]]
    # out = torch.diag(logs[:,targets])  # one-liner instead of loop
    return -out  # negative out
hand_coded = coded_nll_loss(predicted, target)
print('hand calc loss', hand_coded) 

input:
	 tensor([[-0.5645,  1.2495, -0.6334, -0.2390,  0.0440],
        [ 1.4635,  0.9462, -0.1852,  1.0748,  0.0887],
        [-0.0203,  0.1721,  1.2250,  0.4618, -0.0793],
        [-1.3893, -1.3045,  0.4068, -1.0897,  0.2183],
        [-1.6089,  0.3793,  1.5308, -0.2762,  1.1116],
        [-0.4438, -1.1459,  0.1339, -2.2597, -0.5465],
        [ 1.4864, -0.5788,  1.4379, -0.0307,  0.2690]])
target:
	 tensor([1, 0, 0, 1, 2, 1, 2])
predicted:
	 tensor([[-2.4240, -0.6100, -2.4929, -2.0985, -1.8154],
        [-1.0004, -1.5177, -2.6490, -1.3890, -2.3751],
        [-2.1101, -1.9176, -0.8647, -1.6279, -2.1690],
        [-2.6710, -2.5862, -0.8749, -2.3714, -1.0635],
        [-3.9196, -1.9314, -0.7798, -2.5869, -1.1991],
        [-1.4685, -2.1705, -0.8908, -3.2843, -1.5712],
        [-0.9535, -3.0187, -1.0020, -2.4707, -2.1709]])
unreduced loss:
	 tensor([0.6100, 1.0004, 2.1101, 2.5862, 0.7798, 2.1705, 1.0020])
mean loss:
	 tensor(1.4656)
hand calc loss tensor([0.6100, 1.0004, 2.1101, 2.5862, 

In [8]:
# input and target, same as above
loss = nn.CrossEntropyLoss()(input, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

unreduced loss:
 tensor([0.6100, 1.0004, 2.1101, 2.5862, 0.7798, 2.1705, 1.0020])
mean loss:
 tensor(1.4656)


## Multi-Label Categorical Classification


In [9]:
input = torch.randn(7, 5)
target = torch.randint(2, (7, 5), dtype=torch.float)
predicted = torch.sigmoid(input)
unreduced_loss = nn.BCELoss(reduction='none')(predicted, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('input:\n', input)
print('target:\n', target)
print('predicted:\n', predicted)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

input:
 tensor([[-0.3418,  1.2407, -0.6257, -0.0898, -0.5176],
        [ 0.6661,  0.7162, -0.2244, -0.2677,  0.9300],
        [-0.1259,  0.1586, -0.8680, -0.5729,  0.0114],
        [ 0.7484,  1.0068,  1.4012, -1.2712,  0.8315],
        [-1.2328,  0.9791,  0.1576, -0.2293, -1.4694],
        [-0.1202,  0.5292,  0.3490,  0.7890,  1.6020],
        [-0.7683,  1.4478, -0.9227, -0.1112, -1.3520]])
target:
 tensor([[0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1.]])
predicted:
 tensor([[0.4154, 0.7757, 0.3485, 0.4776, 0.3734],
        [0.6606, 0.6718, 0.4441, 0.4335, 0.7171],
        [0.4686, 0.5396, 0.2957, 0.3606, 0.5028],
        [0.6788, 0.7324, 0.8024, 0.2191, 0.6967],
        [0.2257, 0.7269, 0.5393, 0.4429, 0.1870],
        [0.4700, 0.6293, 0.5864, 0.6876, 0.8323],
        [0.3168, 0.8097, 0.2844, 0.4722, 0.2055]])
unreduced loss:
 tensor

In [10]:
# inputs and targets ramins the same as above
unreduced_loss = nn.BCEWithLogitsLoss(reduction='none')(input, target)
mean_loss = nn.BCELoss(reduction='mean')(predicted, target)
print('unreduced loss:\n', unreduced_loss)
print('mean loss:\n', mean_loss)

unreduced loss:
 tensor([[0.5368, 1.4947, 0.4285, 0.6492, 0.4675],
        [0.4146, 0.3978, 0.8116, 0.5682, 1.2625],
        [0.6322, 0.6170, 1.2185, 0.4472, 0.6989],
        [1.1358, 1.3182, 1.6214, 1.5184, 1.1929],
        [1.4886, 0.3189, 0.6175, 0.5850, 1.6764],
        [0.6349, 0.9924, 0.5338, 1.1635, 0.1836],
        [0.3810, 1.6589, 0.3346, 0.7503, 1.5821]])
mean loss:
 tensor(0.8667)


In [ ]:
assert torch.equal(torch.tensor([1, 2]), torch.tensor([1, 2]))